In [1]:
import numpy as np
import scipy.sparse as sp
import networkx as nx
import pandas as pd
from texttable import Texttable
from tqdm import tqdm
from networkx.generators.atlas import *

# Read dataset 

Import original dataset for EDA from:
 http://psb.stanford.edu/psb-online/proceedings/psb18/agrawal.pdf

 Disease pathways have the power to illuminate molecular mechanisms but their discovery is a challenging computational task. It involves identifying all disease-associated proteins, grouping the proteins into a pathway, and analyzing how the pathway is connected to the disease at molecular and clinical levels.

Broadly, a disease pathway in the PPI network is a system of interacting proteins whose atypical activity collectively produces some disease phenotype. 

Methods for disease protein discovery predict candidate disease proteins using the PPI network and known proteins associated with a specific disease. Predicted disease proteins can be grouped into a disease pathway to study molecular disease mechanisms.



### Protein-disease associations

A protein-disease association is a tuple (u, d) indicating that alteration of protein . is linked to disease .. Protein-disease associations are pulled from DisGeNET, a platform that centralized the knowledge on Mendelian and complex diseases. We examine over 21,000 protein-disease associations, which are split among the 519 diseases that each has at least 10 disease proteins. The diseases range greatly in complexity and scope; the median number of associations per disease is 21, but the more complex diseases, e.g., cancers, have hundreds of associations.

In [2]:
# Protein-disease associations
df_assoc = pd.read_csv('./data/bio-pathways-associations.csv')
df_assoc.head(2)

,Disease ID,Disease Name,Associated Gene IDs
0,C0036095,Salivary Gland Neoplasms,"1462, 1612, 182, 2011, 2019, 2175, 2195, 23209..."
1,C0033941,"Psychoses, Substance-Induced","135, 1636, 207, 2099, 2912, 2950, 3350, 3362, ..."


In [3]:
print(f'The number of disease is {len(df_assoc.index)}')

The number of disease is 519


### Disease categories

Diseases are subdivided into categories and subcategories using the Disease Ontology. The diseases in the ontology are each mapped to one or more Unified Medical Language System (UMLS) codes, and of the 519 diseases pulled from DisGeNET, 290 have a UMLS code that maps to one of the codes in the ontology. For the purposes of this study, we examine the second-level of the ontology; this level consists of 10 categories, such as cancers (68 diseases), nervous system diseases (44), cardiovascular system diseases (33), and immune system diseases (21).

In [4]:
# Disease ID - Name - Class
df_dc = pd.read_csv('./data/bio-pathways-diseaseclasses.csv')
size = len(df_dc['Disease Class'].unique())
print(df_dc.head(2))
print('\n')
print(df_dc['Disease Class'].unique())
print('\n')
print(f'The number of disease class is {size}')

  Disease ID              Disease Name                  Disease Class
0   C0023903           Liver neoplasms                         cancer
1   C0018798  Congenital Heart Defects  cardiovascular system disease


['cancer' 'cardiovascular system disease' 'acquired metabolic disease'
 'respiratory system disease' 'immune system disease'
 'integumentary system disease' 'sleep disorder' 'urinary system disease'
 'orofacial cleft' 'gastrointestinal system disease'
 'substance-related disorder' 'polycystic ovary syndrome'
 'nervous system disease' 'bacterial infectious disease'
 'monogenic disease' 'musculoskeletal system disease' 'benign neoplasm'
 'inherited metabolic disorder' 'parasitic infectious disease'
 'viral infectious disease' 'sudden infant death syndrome'
 'endocrine system disease' 'congenital nervous system abnormality'
 'developmental disorder of mental health' 'psoriatic arthritis'
 'cognitive disorder' 'chromosomal disease' 'reproductive system disease'
 'hypospadias' 'cili

### Proximity of disease proteins in the PPI network

Several features, detail found in the original paper


In [5]:
# Disease id network features
df_f = pd.read_csv('./data/bio-pathways-features.csv')
df_f.head(2)

,Disease ID,Disease Name,Size of largest pathway component,Density of pathway,Network Modularity,Distance of Pathway Components,Spatial Network Association
0,C0036095,Salivary Gland Neoplasms,0.088889,0.019192,-0.006214,2.925300,0.404333
1,C0033941,"Psychoses, Substance-Induced",0.352941,0.117647,-0.008137,2.840909,0.256645


### Other data



*  List of interactions between genes
*  List of motifs from disease
*  List of motifs from genes


The analysis of higher-order PPI network structure can be formalized by counting network motifs, which are subgraphs that recur within a larger network. We here focus on graphlets connected non-isomorphic induced subgraphs.

There are 30 possible graphlets of size 2 to 5 nodes. The simplest graphlet is just two nodes connected by an edge, and the most complex graphlet is a clique of size 5. By taking into account the symmetries between nodes in a graphlet, there are 73 different positions or orbits for 2–5-node graphlets, numerated from 0 to 72. For each node in the PPI network we count the number of orbits that the node touches. Motif signature of a protein is thus a set of 73 numbers, hi (. = 0, 1, …, 72) representing the number of induced subgraphs the corresponding node is in, in which the node took the .-th orbital position. We use this signature to represent protein’s higher-order connectivity in the PPI network.


In [6]:
# Edge list between gene
df_n = pd.read_csv('./data/bio-pathways-network.csv')
print(df_n.head(2))
print('\n')
print(f'The number of gene interaction is {len(df_n.index)}')

   Gene ID 1  Gene ID 2
0       1394       2778
1       6331      17999


The number of gene interaction is 342353


In [7]:
# Disease ID to motifs 
df_dm = pd.read_csv('./data/bio-pathways-diseasemotifs.csv')

# Gene ID to motifs 
df_pm = pd.read_csv('./data/bio-pathways-proteinmotifs.csv')

print(f'The number of gene is {len(df_pm.index)}')

The number of gene is 22552


# Motifs detection 


In [8]:
# Utility functions

def load_graph(graph_path):
    """
    Reading an egde list csv as an NX graph object.
    :param graph_path: Path to the edgelist.
    :return graph: Networkx Object.
    """
    graph = nx.from_edgelist(pd.read_csv(graph_path).values.tolist())
    graph.remove_edges_from(nx.selfloop_edges(graph))
    return graph

class MotifCounterMachine(object):
    """
    Connected motif orbital role counter.
    """
    def __init__(self, graph, graphlet_size, output):
        """
        Creating an orbital role counter machine.
        :param graph: NetworkX graph.
        :param args: Arguments object.
        """
        self.graph = graph
        self.output = output
        self.graphlet_size = graphlet_size

    def create_edge_subsets(self):
        """
        Enumerating connected subgraphs with size 2 up to the graphlet size.
        """
        print("\nEnumerating subgraphs.\n")
        self.edge_subsets = dict()
        subsets = [[edge[0], edge[1]] for edge in self.graph.edges()]
        self.edge_subsets[2] = subsets
        unique_subsets = dict()
        for i in range(3, self.graphlet_size+1):
            print("Enumerating graphlets with size: " +str(i) + ".")
            for subset in tqdm(subsets):
                for node in subset:
                    for neb in self.graph.neighbors(node):
                        new_subset = subset+[neb]
                        if len(set(new_subset)) == i:
                            new_subset.sort()
                            unique_subsets[tuple(new_subset)] = 1
            subsets = [list(k) for k, v in unique_subsets.items()]
            self.edge_subsets[i] = subsets
            unique_subsets = dict()

    def enumerate_graphs(self):
        """
        Creating a hash table of the benchmark motifs.
        """
        graphs = graph_atlas_g()
        self.interesting_graphs = {i: [] for i in range(2, self.graphlet_size+1)}
        for graph in graphs:
            if graph.number_of_nodes() > 1 and graph.number_of_nodes() < self.graphlet_size+1:
                if nx.is_connected(graph):
                    self.interesting_graphs[graph.number_of_nodes()].append(graph)

    def enumerate_categories(self):
        """
        Creating a hash table of benchmark orbital roles.
        """
        main_index = 0
        self.categories = dict()
        for size, graphs in self.interesting_graphs.items():
            self.categories[size] = dict()
            for index, graph in enumerate(graphs):
                self.categories[size][index] = dict()
                degrees = list(set([graph.degree(node) for node in graph.nodes()]))
                for degree in degrees:
                    self.categories[size][index][degree] = main_index
                    main_index = main_index + 1
        self.unique_motif_count = main_index

    def setup_features(self):
        """
        Counting all the orbital roles.
        """
        print("\nCounting orbital roles.\n")
        self.features = {node: {i:0 for i in range(self.unique_motif_count)}for node in self.graph.nodes()}
        for size, node_lists in self.edge_subsets.items():
            graphs = self.interesting_graphs[size]
            for nodes in tqdm(node_lists):
                sub_gr = self.graph.subgraph(nodes)
                for index, graph in enumerate(graphs):
                    if nx.is_isomorphic(sub_gr, graph):
                        for node in sub_gr.nodes():
                            self.features[node][self.categories[size][index][sub_gr.degree(node)]] += 1
                        break

    def create_tabular_motifs(self):
        """
        Creating a table with the orbital role features.
        """
        print("Saving the dataset.")
        self.binned_features = {node: [] for node in self.graph.nodes()}
        self.motifs = [[n]+[self.features[n][i] for i in  range(self.unique_motif_count)] for n in self.graph.nodes()]
        self.motifs = pd.DataFrame(self.motifs)
        self.motifs.columns = ["id"] + ["role_"+str(index) for index in range(self.unique_motif_count)]
#         self.motifs.to_csv(self.output, index=None)

    def extract_features(self):
        """
        Executing steps for feature extraction.
        """
        self.create_edge_subsets()
        self.enumerate_graphs()
        self.enumerate_categories()
        self.setup_features()
        self.create_tabular_motifs()

In [9]:
# Load PPI network
graph_path = './data/bio-pathways-network.csv'
G = load_graph(graph_path)

In [10]:
# for row in df_assoc.itertuples():
#     nodes = [int(node) for node in row._3.split(', ')]
#     G_sub = G.subgraph(nodes)
    
#     model1 = MotifCounterMachine(G_sub, 4, 'diseased_orbitals.csv')
#     model1.extract_features()
#     break

In [11]:
dc_dict = {}
for row in df_assoc.itertuples():
    d_id = row._1
    nodes = {int(node) for node in row._3.split(', ')}
    df = df_dc[df_dc['Disease ID'] == d_id]
    if df.empty:
        continue
    disease_class = df['Disease Class'].item()
    if disease_class not in dc_dict:
        dc_dict[disease_class] = nodes
    else:
        dc_dict[disease_class] |= nodes

In [12]:
df_appended = []
for key, value in dc_dict.items():
    nodes = list(value)
    G_sub = G.subgraph(nodes)
    
    model1 = MotifCounterMachine(G_sub, 4, 'diseased_orbitals.csv')
    df_appended.append(model1.extract_features())


Enumerating subgraphs.

Enumerating graphlets with size: 3.


100%|██████████████████████████████████████████████████████████████████████████████| 214/214 [00:00<00:00, 8918.11it/s]


Enumerating graphlets with size: 4.


100%|██████████████████████████████████████████████████████████████████████████████| 858/858 [00:00<00:00, 5958.70it/s]



Counting orbital roles.



...

Enumerating graphlets with size: 4.


100%|██████████████████████████████████████████████████████████████████████| 1030579/1030579 [15:48<00:00, 1086.23it/s]



Counting orbital roles.



 18%|███████████▏                                                  | 14901881/82971742 [24:01:08<109:42:56, 172.34it/s]


KeyboardInterrupt: 

In [ ]:
model1.motifs